# Notebook com a retirada de todos os Ticker de todas as empresas 


In [27]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import time
from IPython.display import clear_output

#Paralelismo 
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed

## Análise de Links e Paralelismo

In [2]:
"""
-------------------------------------------------------------------------------
Função Base
Nesse caso uma função que recebe uma URL e devolve um Bs4
-------------------------------------------------------------------------------
"""
    
def url_bs4(url):

    # Formando o Header 
    header = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest"}
        
    # Testando a URL 
    try:
        #print(f'Analisando a página {url}')
        page = requests.get(url, headers = header, timeout=15)
        
        # Se a página carregou mas não deu 200 mostra o erro e Fala que o Soup é Falso
        if page.status_code != 200:
            print(f'A requisição da página {url} retornou o erro {page.status_code}')
            soup = f'{url} False'
            
        # Se o retorno foi OK = 200, cria o BS4
        else:
            soup_evalue = f" {url.split('/')[-1]}  " + BeautifulSoup(page.text, 'lxml').h2.get_text() #!!!!! Tá tirando só o title !!!! 
            
            if '-' not in soup_evalue:
                soup = soup_evalue.strip().upper() + ' = False'
            else:
                soup = BeautifulSoup(page.text, 'lxml').h2.get_text()
                
            print(f'Avaliando {soup_evalue}')
            clear_output(wait=True)
            
    except Exception as erro:
        print(f'Ao tentar pegar a URL tivemos o erro [{erro}]')
        soup = f'{url} False'
        
    
    # Descansa 10 segundos entre as calls         
    time.sleep(10)
    
    return soup

"""
-------------------------------------------------------------------------------
Função de Paralelização 
-------------------------------------------------------------------------------
"""
   
def main(workers, lista_links):
    
    
    # Criando uma função que vai Criar o Pool de trabalhadores 
    with ThreadPoolExecutor(max_workers = workers) as executor:
        
        """ Criamos um grupo de trabalho(futures) para cada um dos trabalhadores com uma 'list comprehension'"""
        # É como se eu tivesse feito um for item in lista_links future_to url = executor...
        future_to_url = [executor.submit(url_bs4, url) for url in lista_links]
        
        """Essa parte do programa faz o seguinte: 
        O as_completed é uma função que captura "O RETORNO DA FUNÇÃO" de cada um do trabalhadores medida que eles completam
        Mas isso não devolve uma lista como no map e sim um interador que precisa ser passado pr aoutra coisa. 
        Assim que eles todos terminam, faz o que quiser com o resultado retornado pela função aplicada"""
        
        for item in as_completed(future_to_url):
            
            try:                     
                retornos_futuros = item.result() # Não pode esquecer o .result, senão ele fica um Objeto Futures
                
                # Testa o conteúdo dos resultados
                if retornos_futuros != None:
                    stocks_info.append(retornos_futuros)
                else:
                    stocks_info.append(0)
            
            except Exception as erro:
                print(f'Essa iteração específica da função não foi possível. Erro {erro}')
            
    # Retorna a lista com os resultados das puxadas BS4 paralelas 
    return stocks_info 

# Tratamento do Dataframe

## Abertura e separação apenas das colunas necessárias

In [28]:
#Criando o DF
df = pd.read_csv("../Stock_Prices/market_beat.csv") 
df_filtered = df.copy()
df_filtered.tail()

,Ticker,Company,MS,MBI,MB,CNN,BS
3724,ZSPH,ZS Pharma Inc,False,False,True,False,False
3725,ZTCOF,ZTE Corp,False,False,True,True,False
3726,ZTCOY,ZTE Corp,False,False,True,True,False
3727,ZURVY,Zurich Insurance Group AG,False,False,True,True,False
3728,ZZLL,ZZLL Information Technology Inc,False,False,True,True,False


## Função que pega cada um dos ticker e transforma ele em um novo DataFrame com um Link

In [15]:
def transforma_link(ticker):
    return f'https://bastter.com/mercado/stock/{ticker}'
    #return f'https://www.cnbc.com/quotes/?symbol={ticker}'
    #return f'https://www.morningstar.com/stocks/xnys/{ticker}/quote'
    #return f'https://www.bing.com/search?q={ticker}+stock'
    #return f'https://www.marketbeat.com/stocks/{ticker}/'

df_filtered['Ticker'] = df_filtered['Ticker'].apply(transforma_link)
df_filtered['Ticker'] = df_filtered['Ticker'].str.lower() 

# Pegando apenas o Ticker
df_filtered = df_filtered.drop(['Company','MS','MBI','MB','CNN'], axis =1)
df_filtered.tail()

,Ticker
21291,https://bastter.com/mercado/stock/top
21292,https://bastter.com/mercado/stock/vaso
21293,https://bastter.com/mercado/stock/vipr
21294,https://bastter.com/mercado/stock/zeon
21295,https://bastter.com/mercado/stock/international


## Função que faz a criação de Uma lista com os links pra ser avalidada pela função recursiva lá em cima

In [16]:
# Cria a lista em cima da Series
list_df = df_filtered.values.tolist()

# Mas como ela gera uma lista dentro de uma lista eu preciso tirar elas pra 1 lista só 
links_stocks = []

for lista in list_df:
    for item in lista: 
        links_stocks.append(item)

In [17]:
# Pra quando precisar fazer teste em cima do dataframe 
teste = links_stocks[:25]
print(teste[:5])

['https://bastter.com/mercado/stock/a', 'https://bastter.com/mercado/stock/aa', 'https://bastter.com/mercado/stock/aaag', 'https://bastter.com/mercado/stock/aaalf', 'https://bastter.com/mercado/stock/aaaly']


# Execução do Programa 

In [18]:
if __name__ == '__main__':
    
    
    links_teste = ['https://bastter.com/mercado/stock/xupilas' , 'https://bastter.com/mercado/stock/mmm', 'https://bastter.com/mercado/stock/aa']
    
    # Lista onde as informações serão armazenadas
    stocks_info = []
    
    # Executo a Função Main jogando como parametro a lista Links 
   
    #%time main(10, links_teste)
    #%time main(50, teste)
    %time main(25, links_stocks)
    
    print('A lista Stocks Info tem todas as informações BS4 das stocks. Filtre-as em outra variável com Lambdas ou Comprehensions')
    

Wall time: 2h 37min 13s
A lista Stocks Info tem todas as informações BS4 das stocks. Filtre-as em outra variável com Lambdas ou Comprehensions


In [19]:
for item in stocks_info[:10]:
  print(item)

AAALY = False
AABA = False
AACEY = False
AAAG = False
AABGF = False
AAGEF = False
AACAY = False
AAALF = False
AAGRY = False
AAHTF = False


# Avaliação dos resultados BS4

## For LOOP
> Pega cada um dos itens da lista Stocks_Info (BS4),e joga numa nova lista [Stocks_Evalueted] contendo os Ticker e Se a ação é negociada.

In [20]:
evalueted_stock = list()

for item in stocks_info:
    
    if ' False' in item :
        evalueted_stock.append((item.split('/')[-1].replace("=","").replace("False","").strip().upper(), False))  # Aplica o ticker_remover no Título da Página
        
    else: # OS BS4 VEM PRA CÁ!!
        evalueted_stock.append((item.split()[0].split()[0], True))

evalueted_stock.sort()
print(evalueted_stock[5000:5020])


[('CUNB', False), ('CUNCF', False), ('CUO', True), ('CUOTF', False), ('CUPHF', False), ('CUPUF', False), ('CUR', True), ('CURLF', False), ('CURM', False), ('CURN', False), ('CURO', True), ('CURR', False), ('CUSI', False), ('CUSPY', False), ('CUTEF', False), ('CUTR', True), ('CUUCF', False), ('CUUHF', False), ('CUVMF', False), ('CUYRF', False)]


## Criação do Dicionário/ Dataframe
> Pega a lista com os tickers do passo anterior e cria um dicionário onde as keys são os Ticker

In [21]:
dicionario_stocks_evalueted = {key.upper() : value for key, value in evalueted_stock}
print(dicionario_stocks_evalueted['AAPL'])

True


In [22]:
df_evalueted = pd.DataFrame.from_dict(dicionario_stocks_evalueted, orient='index')
df_evalueted = df_evalueted.reset_index()
df_evalueted = df_evalueted.rename(columns = {"index":"Ticker", 0: "BS"}) 
df_evalueted

,Ticker,BS
0,A,True
1,AA,True
2,AAAG,False
3,AAALF,False
4,AAALY,False
...,...,...
21246,^HSI,False
21247,^IXIC,False
21248,^N225,False
21249,^OMXSPI,False


## Merge dos DFs

In [26]:
df_merged = pd.merge(df, df_evalueted, on='Ticker', how='outer')
df_merged.to_csv('bastter_all_stocks.csv')
